# Simple `fython` Demo

## Fortran

The following Fortran module contains data that will be exchange between Python and Fortran.

In [16]:
# %load data_mod.f90
module mod_data

! Data exange module
! The data declares in this module will be exchanged between Fortran
! and Python.

    implicit none
    ! Keep these values over the whole program run.
    save
    real::  number1 = 0.0
    integer:: number2 = 1

end module mod_data


This Fortran subroutine will be callable from Python.

In [8]:
# %load increase.f90
subroutine increase()

    ! Access to the shared data, i.e. the those also accessible from Python.
    use mod_data
    implicit none
    ! Keep these values over the whole program run.
    save

    ! Increment values at each call.
    ! These changes will show up in Python.
    number1 = number1 + 10.0
    number2 = number2 + 1

end subroutine increase


This is the main Fortran program converted into a subroutine.
It takes a Python callable `cback` as argument.
It calls `cback` several times:

In [7]:
# %load callback.f90

! Subroutine that will use a callback function from Python

subroutine callback_test(cback)

    implicit none

    ! Declare the Python function as `external`
    external cback

    integer k

    do k = 1, 3
        write(*,*) '### FORTRAN handing over to Python'
        ! This hand the control over to Python
        call cback()
        write(*,*) '### back in FORTRAN'
    enddo
end


## Compile

We use `f2py`, which is a part of NumPy to compile the above subroutine into a Python extension:

In [9]:
# %load Makefile
callback:
	f2py --overwrite-signature callback.f90 increase.f90 data_mod.f90 -m fython_test -h fython_test.pyf
	f2py -c fython_test.pyf callback.f90 data_mod.f90 increase.f90


We do this in to steps. First, we create a an interface file `fython_test.pyf`: 

In [17]:
# %load fython_test.pyf
!    -*- f90 -*-
! Note: the context of this file is case sensitive.

python module callback_test__user__routines 
    interface callback_test_user_interface 
        subroutine cback ! in :fython_test:callback.f90:callback_test:unknown_interface
        end subroutine cback
    end interface callback_test_user_interface
end python module callback_test__user__routines
python module fython_test ! in 
    interface  ! in :fython_test
        subroutine callback_test(cback) ! in :fython_test:callback.f90
            use callback_test__user__routines
            external cback
        end subroutine callback_test
        subroutine increase ! in :fython_test:increase.f90
            use mod_data
        end subroutine increase
        module mod_data ! in :fython_test:data_mod.f90
            real, optional :: number1=0.0
            integer, optional :: number2=1
        end module mod_data
    end interface 
end python module fython_test

! This file was auto-generated with f2py (version:2).
! See http://cens.ioc.ee/projects/f2py2e/


It is possible to edit this file to change the behavior of the extension. Most of the time this is not necessary and the generated `pyf` file is fine as it is.

The second line in Makefile compiles an extension, i.e. a `*.pyd` on WIndows or a `*.so` on Linux/Mac.

## Python

Now we can import our extension and use it:

In [10]:
# %load run.py
"""Simple Fython demo
"""

import fython_test


class Func:
    # pylint: disable=too-few-public-methods
    """Class whose instances act like a function, i.e. are callables

    https://docs.python.org/3/reference/datamodel.html?highlight=__call__#object.__call__

    Called when the instance is “called” as a function; if this method is
    defined, `x(arg1, arg2, ...)` is a shorthand for
    `x.__call__(arg1, arg2, ...).`


    """

    def __init__(self, val):
        self.val = val
        self.counter = 0

    def __call__(self):
        self.counter += 1
        print(f'>>> Python: Called {self.counter} time')
        print(f'>>> Python: Value {self.val}')
        print(f'>>> Python: Number 1 in FORTRAN {fython_test.mod_data.number1}')
        print(f'>>> Python: Number 2 in FORTRAN {fython_test.mod_data.number2}')
        fython_test.increase()


if __name__ == '__main__':
    fython_test.callback_test(Func(10))


Python has the concept of a callable.
Often this is a function.
But it is also possible to use a class with a special method `__call__`.
An instance of such a class will be a callable, that behaves just like a function but allows for a state via attributes of `self`. 

## Use

Now we run our program:

In [5]:
!python run.py

 ### FORTRAN handing over to Python
>>> Python: Called 1 time
>>> Python: Value 10
>>> Python: Number 1 in FORTRAN 0.0
>>> Python: Number 2 in FORTRAN 1
 ### back in FORTRAN
 ### FORTRAN handing over to Python
>>> Python: Called 2 time
>>> Python: Value 10
>>> Python: Number 1 in FORTRAN 10.0
>>> Python: Number 2 in FORTRAN 2
 ### back in FORTRAN
 ### FORTRAN handing over to Python
>>> Python: Called 3 time
>>> Python: Value 10
>>> Python: Number 1 in FORTRAN 20.0
>>> Python: Number 2 in FORTRAN 3
 ### back in FORTRAN
